In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import pandas and create the necessary data frame
df= pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')
df.head()

oh my! this dataset looks kinda scary....but is fairly easy, jus needs a bit more work and concentration...

POWERING UP!!!
lets goooooo!!!

In [ ]:
df.columns

In [ ]:
#lets check for missing values
df.isnull().sum()

A mushroom can either be classified as edible('e') or posionous('p)
so with the class column being our response column...lets take it out n assign it a variable 'y'

In [ ]:
y = df['class']
y

In [ ]:
x= df.drop(['class'],axis=1)
x

we have our features...now we need to check the corralations but first we have to change the letters to categorical numbers...

In [ ]:
mushroom_class = {
    'e':0,
    'p':1
}
y = y.map(mushroom_class)
y

for the mushroom class it was fairy easy...now time to eat some hard jerky n do the same for the features

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder() 
for features in x.columns:
    x[features] = encoder.fit_transform(x[features])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.subplots(figsize=(18,18))
sns.heatmap(x.corr(), annot=True, linewidths=.5, fmt= '.1f')

appearently veil-type is full of zero's , lets drop it

In [ ]:
x = x.drop(['veil-type'], axis=1)


Am not yet good at visualizing data...forgive me bout that.
But for the coming notebooks i can say ill be pro...

In [ ]:
plt.subplots(figsize=(18,18))
sns.heatmap(x.corr(), annot=True, linewidths=.5, fmt= '.1f')

for feature selection...Lets use RFE to select the best features, because my human judgement may be flawed.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=42)
clf = RandomForestClassifier()      
rfecv = RFECV(estimator=clf, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(x_train, y_train)

print('Optimal number of features :', rfecv.n_features_)
print('Best features :', x_train.columns[rfecv.support_])

beautiful, with these features we'll be able to create a good model

In [ ]:
x1 = x[['odor','gill-size','gill-color','stalk-root','ring-type','spore-print-color','population']]
x1

In [ ]:
#for the type of classification model, we'll try most of them to find out which provides the best outcome
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

x_train, x_test,y_train, y_test = train_test_split(x1, y, test_size=.3, random_state=42)


logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict(x_test)
#lets look at its accuracy
acc = accuracy_score(y_test, y_pred_logreg)
acc

In [ ]:
#lets try random forest

rfclf = RandomForestClassifier(n_estimators=1000, criterion='entropy')
rfclf.fit(x_train, y_train)
rf_y_pred = rfclf.predict(x_test)
acc = accuracy_score(y_test,rf_y_pred)
acc

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtclf = DecisionTreeClassifier(criterion='entropy')
dtclf.fit(x_train, y_train)
dt_pred = dtclf.predict(x_test)
acc = accuracy_score(y_test, dt_pred)
acc


In [ ]:
from sklearn.svm import SVC

acc_svc = []
for s in range(1,11):
  model = SVC(C=s)
  model.fit(x_train,y_train)
  model_pred = model.predict(x_test)
  acc = accuracy_score(y_test, model_pred)
  acc_svc.append(acc)

acc_svc

So we've tried various classification algorithms each providing their own accuracy.

1.LOGISTIC REGRESSION : 89.9%

2.RANDOM FOREST: 100%

3.SVM with a regulization of 9: 99.75%

4.DECISION TREE: 100%


hope this hepls even if its a bit, and if you have any adjustments, or if i made mistakes, please comment.
Thank you!
happy coding, 
:)